In [1]:
EN_train = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\EN\train"
EN_modified = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\EN\modified_train"
EN_test = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\EN\dev.in"
EN_output = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\EN\dev.p2.out"
EN_gold = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\EN\dev.out"
EN_viterbi = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\EN\dev.p3.out"
EN_test1 = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\EN\test.in"

CN_train = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\CN\train"
CN_modified = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\CN\modified_train"
CN_test = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\CN\dev.out"
CN_output = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\CN\dev.p2.out"
CN_gold = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\CN\dev.out"
CN_viterbi = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\CN\dev.p3.out"


SG_train = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\SG\train"
SG_modified = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\SG\modified_train"
SG_test = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\SG\dev.in"
SG_output = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\SG\dev.p2.out"
SG_gold = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\SG\dev.out"
SG_viterbi = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\SG\dev.p3.out"


FR_train = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\FR\train"
FR_modified = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\FR\modified_train"
FR_test = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\FR\dev.in"
FR_output = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\FR\dev.p2.out"
FR_gold = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\FR\dev.out"
FR_viterbi = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\FR\dev.p3.out"
FR_test1 = r"C:\Users\Matthew\Desktop\SUTD\Term 6\Machine Learning\ML Project\FR\test.in"

In [ ]:
#part 5
from collections import Counter
from operator import itemgetter
import copy, subprocess, os

def remove_tag(training_file): # generate train_no_tag file
    file = open(training_file,"r",encoding="utf8")
    new_file = training_file + " without tags"
    output_file = open(new_file,"w+",encoding="utf8")
    for line in file:
        line = line.split()
        if len(line)>1:
            output_file.write(line[0]+"\n")
        else:
            output_file.write("\n")
    file.close()
    output_file.close()
    return

def para(training_file): # output a dictionary of emission_parameter
    c_y = Counter() # count the y tags
    e_para = Counter() # count the y|x tags
    t_para = Counter()
    state = "start"
    y0_y1 = ""
    # init_e_para = Counter()
    # init_t_para = Counter()
    file = open(training_file, "r", encoding='utf8')
    for line in f:
        line = line.rstrip('\n')
        if (len(line) == 0):
            y0_y1 = state + " stop"
            state = "stop"
        else:
            e_para[line]+=1
            line = line.split()[1]
            y0_y1 = state + " " + line
            state = line
            c_y[line]+=1
        t_para[y0_y1]+=1
    for tag in c_y:
        e_para["#UNK#"+tag]=1
    file.close()
    return  e_para, t_para

def init_para(training_file):
    c_y = Counter() # count the y tags
    e_para = Counter() # count the y|x tags
    t_para = Counter()
    state = "start"
    y0_y1 = ""
    file = open(training_file, "r", encoding='utf8')
    for line in file:
        line = line.rstrip('\n')
        if (line == ""):
            y0_y1 = state + " stop"
            state = "stop"
        else:
            e_para[line]=0
            line = line.split()[1]
            y0_y1 = state + " " + line
            state = line
            c_y[line]=0
        t_para[y0_y1]=0
    for tag in c_y:
        e_para["new_word "+tag]=0
    file.close()
    return  e_para, t_para

def test_phase_perceptron(test_file,emission_parameter,transition_parameter,top_k=1, run = 1):
    tags = ["O","B-neutral","B-positive","B-negative","I-neutral","I-positive","I-negative"]
    step_k = 1
    v_label = "O"
    u_label = "start"
    optimal_value = -1
    pi = [step_k,v_label,optimal_value,u_label]
    sentence = []
    output = []
    list_u = [0,1,2,3,4,5,6]
    dict_u = {}
    state = (step_k, list_u)
    data_x = set()
    for entry in emission_parameter.keys():
        entry = entry.split(' ')[0]
        data_x.add(entry)
    file_dir = test_file[:-3]+str(int(test_file[-3:])+run)
    file = open(test_file, "r", encoding='utf8')
    output_file = open(file_dir, "w+", encoding='utf8')
    for line in file:
        line = line.rstrip()
        if line!="":
            for id_v,v in enumerate(tags):
                best = -1
                best_add = -1
                for id_u,u in enumerate(state[1]): #enumerate list_u stored in state
                    if(line in data_x):
                        b_word = line + " " + v
                        b_value = emission_parameter[b_word]
                    else:
                        b_word = "new_word " + v
                        b_value = emission_parameter[b_word]

                    if state[0] == 1:
                        u_label = "start"
                        uv_label = u_label+" " + v
                        a_uv_value = transition_parameter[uv_label]
                        optimal_value = a_uv_value+b_value
                    else:
                        u_label = u[1]
                        uv_label = u_label + " " + v
                        a_uv_value = transition_parameter[uv_label]
                        optimal_value =  u[2] + a_uv_value + b_value
                    if optimal_value>best:
                        best = optimal_value
                        pi = [step_k, v, best, u_label]
                dict_u[(step_k,v)] = pi[3]
                list_u[id_v] = pi
            sentence.insert(0,line)
            list_u2 = copy.deepcopy(list_u)
            step_k+=1
            state = (step_k,list_u2)
        else: # when it reaches the empty line
            best = -1
            for id_u,u in enumerate(state[1]):
                uv_label = u[1] + " stop"
                a_uv_value = transition_parameter[uv_label]
                optimal_value = u[2]+a_uv_value
                pi = [state[0],"stop",optimal_value,u[1]]
                list_u[id_u] = pi
            list_u = sorted(list_u,key=itemgetter(2),reverse=True)  # arrange by descending order to choose top_k
            u_label = list_u[top_k-1][3]
            output.append(sentence[0]+" "+u_label+"\n")
            for i in range(1,len(sentence)): # back tracking to find the y_labels
                search_key = (step_k-i,u_label)
                u_label = dict_u[search_key]
                output.insert(0,sentence[i]+" "+u_label+"\n")
            for i in output:
                output_file.write(i)
            output_file.write("\n")
            sentence = []
            output=[]
            dict_u={}
            step_k=1
            state = (step_k,[0,1,2,3,4,5,6])
    output_file.close()
    return

def update_para(training_file):
    parameter1 = para(training_file)
    e_c1 = copy.deepcopy(parameter1[0])
    t_c1 = copy.deepcopy(parameter1[1])
    init_parameter = init_para(training_file) #initialize epara and tpara to 0
    e_para = copy.deepcopy(init_parameter[0])
    t_para = copy.deepcopy(init_parameter[1])
    remove_tag(training_file) # generate training file without tags
    blank_file = training_file + " without tags"
    print("Running %d of %d:" %(run_num+1,n) )
    test_phase_perceptron(blank_file,e_para,t_para,run = run_num+1) # generates prediction file
    predicted_file = "dev.p5.out"
    parameter2 = para(predicted_file) # to get e_c2, t_c2
    e_c2 = copy.deepcopy(parameter2[0])
    t_c2 = copy.deepcopy(parameter2[1])
    e_para += e_c1  
    e_para.subtract(e_c2)
    t_para += t_c1
    t_para.subtract(t_c2)
    proc = subprocess.Popen(["ls"], stdout=subprocess.PIPE, shell=True)
    (out,err) = proc.communicate()
    return e_para ,t_para

parameter = update_para(EN_test1, n = 1)